In [2]:
import re
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import datetime
import warnings

import torch
from torch.optim import Adagrad
from sklearn.metrics import log_loss, roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from utils import find_outliers_by_3segama


# sys.path.append("/Users/wzq/Desktop/game")
# from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
# from deepctr_torch.models import *
# from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint

# jupyter配置
from IPython.display import display
pd.options.display.max_rows=10000 #Notebook 的一个cell的显示行数
pd.options.display.max_columns=100000#Notebook 的一个cell的显示列数
pd.set_option('display.max_colwidth', None)

In [9]:
data_train = pd.read_csv("./data/train.csv")
data_test = pd.read_csv("./data/testA.csv")
# data = pd.concat([data_train,data_test], axis=0)
print("train_size:", data_train.shape,
      "test_size:", data_test.shape,)

number_cols = ["loanAmnt", "term", "interestRate", "installment", "annualIncome",
               "dti", "delinquency_2years", "ficoRangeLow", "ficoRangeHigh", "openAcc", "pubRec", 
               "pubRecBankruptcies", "revolBal", "revolUtil", "totalAcc", "employmentLength"] + ["n{}".format(i) for i in range(15)]
time_cols = ["issueDate", "earliesCreditLine"]
sparse_features = ['grade', 'subGrade', 'homeOwnership', 'verificationStatus', 'purpose', "postCode", 'regionCode','employmentTitle', "title"]

# 去掉 'years' 或 'year' 字样，处理 '10+ years'
def clean_experience(value):
    if pd.isna(value):  # 处理 NaN
        return np.nan
    value = value.replace('years', '').replace('year', '').strip()
    if '+' in value:  # 处理 10+ years
        return int(value.replace('+', ''))
    if '<' in value:
        return 0
    return int(value)

for data in [data_train, data_test]:
    data['employmentLength'] = data['employmentLength'].apply(clean_experience)
    # 处理空值
    # data[number_cols] = data[number_cols].fillna(0)
    # data[category_cols] = data[category_cols].fillna(-1)
    #按照中位数填充数值型特征
    data[number_cols] = data[number_cols].fillna(data[number_cols].median())
    #按照众数填充类别型特征
    data[sparse_features] = data[sparse_features].fillna(data[sparse_features].mode().iloc[0])

    # 时间处理
    data['issueDate'] = pd.to_datetime(data['issueDate'])
    # 拆分特征
    data['issueDate_year'] = data['issueDate'].dt.year
    data['issueDate_month'] = data['issueDate'].dt.month
    # data['issueDate_day'] = data['issueDate'].dt.day 都是1日
    data['issueDate_weekday'] = data['issueDate'].dt.weekday  # 周一为0，周日为6
    data['issueDate_is_weekend'] = (data['issueDate_weekday'] >= 5).astype(int)

    data["earliesCreditLine"] = pd.to_datetime(data["earliesCreditLine"], format="%b-%Y")
    data["earliesCreditLine_year"] = data["earliesCreditLine"].dt.year
    data["earliesCreditLine_month"] = data["earliesCreditLine"].dt.month
    display(data.head(5))
    display("空值和时间类型数据处理done:", data.shape)


    # 编码
    # 对稀疏的类别特征进行简单的处理，处理为连续整数类别
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
    # # 类型数在2之上，又不是高维稀疏的,且纯分类特征
    # data = pd.get_dummies(data, columns=['subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode'], drop_first=True, dtype=np.uint8)
    # # 高维类别特征需要进行转换
    # for f in ['employmentTitle', 'postCode', 'title']:
    #     data[f+'_cnts'] = data.groupby([f])['id'].transform('count')
    #     data[f+'_rank'] = data.groupby([f])['id'].rank(ascending=False).astype(int)
    #     del data[f]
    display("对类别数据进行编码 done", data.head(5))

    # 特征交叉
    
    for feat in ["n{}".format(i) for i in range(15)] + ["issueDate_year", "issueDate_month", "issueDate_weekday", "issueDate_is_weekend","earliesCreditLine_year", "earliesCreditLine_month"]:
        data['grade_to_mean_' + feat] = data.groupby([feat])['grade'].transform('mean')
        data['grade_to_std_' + feat] = data.groupby([feat])['grade'].transform('std')
        # if 'grade_to_mean_' + feat not in number_cols:
        #     number_cols.extend(['grade_to_mean_' + feat, 'grade_to_std_' + feat])
    print("特征交叉 done")

# 类别字段
for col in sparse_features: 
    temp_dict = data_train.groupby([col])['isDefault'].agg(['mean']).reset_index().rename(columns={'mean': col + '_target_mean'})
    temp_dict.index = temp_dict[col].values
    temp_dict = temp_dict[col + '_target_mean'].to_dict()

    data_train[col + '_target_mean'] = data_train[col].map(temp_dict)
    data_test[col + '_target_mean'] = data_test[col].map(temp_dict)
    # if col + '_target_mean' not in number_cols:
    #     number_cols.append(col + '_target_mean')
data = pd.concat([data_train,data_test], ignore_index=True)
print("特征交叉 \n",
      "train_size:", data_train.shape,
      "test_size:", data_test.shape,
      "data_process_size:", data.shape)




# 删除无用数据
data = data.drop(["id", "earliesCreditLine", "policyCode", "issueDate"], axis=1)
display(data.head(5))
display("删除无用特征 done: ", data.shape)

# 删除异常数据（基于3个标准差）【删除】
# features = [f for f in data.columns if f not in ["isDefault"]]
# train_data = data.iloc[:800000, :]
# test_data = data.iloc[800000:, :]
# for fea in features:
#     train_data = find_outliers_by_3segama(train_data,fea)
#     # train_data = train_data[train_data[fea+'_outliers']=='正常值']
#     # train_data = train_data.reset_index(drop=True)
#     # train_data = train_data.drop(columns=[fea+'_outliers'])
# # train_data = train_data.reset_index(drop=True)
# data_process = pd.concat([train_data,test_data], ignore_index=True)

# print("process done \n",
#       "train_size:", train_data.shape,
#       "test_size:", test_data.shape,
#       "data_process_size:", data_process.shape)



train_size: (800000, 47) test_size: (200000, 46)
空值和时间类型数据处理


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDate_year,issueDate_month,issueDate_weekday,issueDate_is_weekend,earliesCreditLine_year,earliesCreditLine_month
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2.0,2,110000.0,2,2014-07-01,1,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,2001-08-01,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,2014,7,1,0,2001,8
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5.0,0,46000.0,2,2012-08-01,0,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,2002-05-01,1723.0,1.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,2012,8,2,0,2002,5
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8.0,0,74000.0,2,2015-10-01,0,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,2006-05-01,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,2015,10,3,0,2006,5
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10.0,1,118000.0,1,2015-08-01,0,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,1999-05-01,4.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,2015,8,5,1,1999,5
4,4,3000.0,3,12.99,101.07,C,C2,54.0,6.0,1,29000.0,2,2016-03-01,0,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,1977-08-01,11.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,2016,3,1,0,1977,8


'时间处理:'

(800000, 53)

对类别数据进行编码


...

/var/folders/h7/wvn6h00n193c_wdchc9p0dfh0000gn/T/ipykernel_88104/3224063680.py:80: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_train[col + '_target_mean'] = data_train[col].map(temp_dict)
/var/folders/h7/wvn6h00n193c_wdchc9p0dfh0000gn/T/ipykernel_88104/3224063680.py:80: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_train[col + '_target_mean'] = data_train[col].map(temp_dict)
/var/folders/h7/wvn6h00n193c_wdchc9p0dfh0000gn/T/ipykernel_88104/3224063680.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is 

特征交叉 
 train_size: (800000, 104) test_size: (200000, 103) data_process_size: (1000000, 104)
删除


,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,title,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDate_year,issueDate_month,issueDate_weekday,issueDate_is_weekend,earliesCreditLine_year,earliesCreditLine_month,grade_to_mean_n0,grade_to_std_n0,grade_to_mean_n1,grade_to_std_n1,grade_to_mean_n2,grade_to_std_n2,grade_to_mean_n3,grade_to_std_n3,grade_to_mean_n4,grade_to_std_n4,grade_to_mean_n5,grade_to_std_n5,grade_to_mean_n6,grade_to_std_n6,grade_to_mean_n7,grade_to_std_n7,grade_to_mean_n8,grade_to_std_n8,grade_to_mean_n9,grade_to_std_n9,grade_to_mean_n10,grade_to_std_n10,grade_to_mean_n11,grade_to_std_n11,grade_to_mean_n12,grade_to_std_n12,grade_to_mean_n13,grade_to_std_n13,grade_to_mean_n14,grade_to_std_n14,grade_to_mean_issueDate_year,grade_to_std_issueDate_year,grade_to_mean_issueDate_month,grade_to_std_issueDate_month,grade_to_mean_issueDate_weekday,grade_to_std_issueDate_weekday,grade_to_mean_issueDate_is_weekend,grade_to_std_issueDate_is_weekend,grade_to_mean_earliesCreditLine_year,grade_to_std_earliesCreditLine_year,grade_to_mean_earliesCreditLine_month,grade_to_std_earliesCreditLine_month,grade_target_mean,subGrade_target_mean,homeOwnership_target_mean,verificationStatus_target_mean,purpose_target_mean,postCode_target_mean,regionCode_target_mean,employmentTitle_target_mean,title_target_mean
0,35000.0,5,19.52,917.97,4,21,292,2.0,2,110000.0,2,1.0,1,137,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,1,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,2014,7,1,0,2001,8,1.697797,1.299334,1.709250,1.277059,1.610733,1.294469,1.610733,1.294469,1.702065,1.301336,1.693955,1.280283,1.722995,1.290977,1.760036,1.270381,1.744015,1.280106,1.615441,1.293735,1.746386,1.256176,1.745286,1.294232,1.744267,1.294101,1.733227,1.296566,1.709323,1.282167,1.853427,1.317010,1.824072,1.298454,1.738670,1.292334,1.740857,1.296685,1.749915,1.295814,1.725343,1.293477,0.384291,0.376903,0.207800,0.237858,0.295190,0.193253,0.211770,0.165923,0.305068
1,18000.0,5,18.49,461.90,3,16,149667,5.0,0,46000.0,2,0.0,0,156,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,1394,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,2012,8,2,0,2002,5,1.697797,1.299334,1.691934,1.292339,1.682598,1.292984,1.682598,1.292984,1.667022,1.336668,1.669991,1.309348,1.651254,1.321469,1.683015,1.306611,1.642824,1.321938,1.683047,1.293067,1.742871,1.302629,1.745286,1.294232,1.744267,1.294101,1.733227,1.296566,1.709323,1.282167,1.591942,1.293269,1.794351,1.314181,1.745863,1.300091,1.740857,1.296685,1.783588,1.291816,1.747614,1.292040,0.303852,0.297572,0.171535,0.237858,0.211370,0.154922,0.159262,0.200000,0.071429
2,12000.0,5,16.99,298.17,3,17,25137,8.0,0,74000.0,2,0.0,0,337,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,2015,10,3,0,2006,5,1.697797,1.299334,1.961515,1.387882,1.651435,1.281073,1.651435,1.281073,2.196146,1.343377,2.504425,1.313295,1.806243,1.334069,1.760036,1.270381,1.902340,1.286098,1.654446,1.279690,1.678111,1.326820,1.745286,1.294232,1.744267,1.294101,1.733227,1.296566,2.077380,1.324700,1.714545,1.301222,1.724798,1.280798,1.757549,1.281157,1.740857,1.296685,1.894051,1.287961,1.747614,1.292040,0.303852,0.304015,0.171535,0.237858,0.211370,0.202395,0.197254,0.000000,0.217977
3,11000.0,3,7.26,340.96,0,3,36207,10.0,1,118000.0,1,0.0,4,148,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,4,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,2015,8,5,1,1999,5,1.893912,1.230311,1.744851,1.280048,1.769022,1.271178,1.769022,1.271178,1.702065,1.301336,1.610821,1.300578,1.735671,1.268465,1.683015,1.306611,1.689353,1.297915,1.770178,1.272207,1.754362,1.274229,1.745286,1.

'删除: '

(1000000, 100)

In [10]:
# data.iloc[:train_size,:].to_csv('./data/train_processed.csv')
# data.iloc[train_size:,:].to_csv('./data/test_processed.csv')
data.to_csv ('./data/data_processed_v9.csv', index=False)


In [98]:
# def find_outliers_by_3segama(data, fea):
#     data_std = np.std(data[fea])
#     data_mean = np.mean(data[fea])
#     outliers_cut_off = data_std * 3
#     lower_rule = data_mean - outliers_cut_off
#     upper_rule = data_mean + outliers_cut_off
#     data = data[(data[fea] >= lower_rule) & (data[fea] <= upper_rule)]
#     # data[fea+'_outliers'] = data[fea].apply(lambda x:str('异常值') if x > upper_rule or x < lower_rule else '正常值')
#     print(data.shape)
#     return data
# data_train = pd.read_csv("./data/train.csv")
# data_test = pd.read_csv("./data/testA.csv")
# data = pd.concat([data_train,data_test], axis=0)
# display(data.shape)
# train_data = data.iloc[:800000, :]
# test_da = find_outliers_by_3segama(train_data,"interestRate")
# test_da.shape

In [104]:
data_process.iloc[800000:,:].isna().sum()

loanAmnt                   0.0
term                       0.0
interestRate               0.0
installment                0.0
grade                      0.0
subGrade                   0.0
employmentTitle            0.0
employmentLength           0.0
homeOwnership              0.0
annualIncome               0.0
verificationStatus         0.0
isDefault                  0.0
purpose                    0.0
postCode                   0.0
regionCode                 0.0
dti                        0.0
delinquency_2years         0.0
ficoRangeLow               0.0
ficoRangeHigh              0.0
openAcc                    0.0
pubRec                     0.0
pubRecBankruptcies         0.0
revolBal                   0.0
revolUtil                  0.0
totalAcc                   0.0
initialListStatus          0.0
applicationType            0.0
title                      0.0
n0                         0.0
n1                         0.0
n2                         0.0
n3                         0.0
n4      